## Reverse the words

### input and target data

In [1]:
import pandas as pd
import numpy as np

In [2]:
data = {
    "input_words" : ['apple', 'ball', 'car', 'dog', 'alex', 'white', 'zebra', 'monkey', 'fight', 'joker', 'van', 'queen'],
    "target_words" : ['elppa', 'llab', 'rac', 'god', 'xela', 'etihw', 'arbez', 'yeknom', 'thgif', 'rekoj', 'nav', 'neeuq']
}

### load a pre trained model

In [3]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Trainer, TrainingArguments
from datasets import Dataset

c:\Users\lenovo\anaconda3\envs\tfenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
dataset = Dataset.from_dict(data)

In [5]:
model_name = 't5-small'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

### tokenize

In [6]:
def preprocessor(data) : 
    model_inputs = tokenizer(
        data['input_words'], padding = 'max_length', truncation = True, max_length = 10
    )

    with tokenizer.as_target_tokenizer() : 
        labels = tokenizer(
            data['target_words'], padding = 'max_length', truncation = True, max_length = 10
        )

    model_inputs['labels'] = labels['input_ids']

    return model_inputs

In [7]:
tokenize_dataset = dataset.map(preprocessor, batched = False)

Map:   0%|          | 0/12 [00:00<?, ? examples/s]

c:\Users\lenovo\anaconda3\envs\tfenv\lib\site-packages\transformers\tokenization_utils_base.py:4007: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
Map: 100%|██████████| 12/12 [00:00<00:00, 763.47 examples/s]


### training the model

In [22]:
training_args = TrainingArguments(
    output_dir = './results',
    per_device_train_batch_size = 2,
    num_train_epochs = 10,
    logging_dir = './logs',
    logging_steps = 1,
    save_strategy = "no"
)

In [23]:
trainer = Trainer(
    model = model, 
    args = training_args,
    train_dataset = tokenize_dataset
)

In [24]:
trainer.train()

c:\Users\lenovo\anaconda3\envs\tfenv\lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss
1,0.877400
2,0.697900
3,0.493800
4,0.243300
5,0.694900
6,0.328000
7,0.393800
8,0.925000
9,0.500400
10,0.545700


TrainOutput(global_step=60, training_loss=0.5131156225999196, metrics={'train_runtime': 19.6389, 'train_samples_per_second': 6.11, 'train_steps_per_second': 3.055, 'total_flos': 317207347200.0, 'train_loss': 0.5131156225999196, 'epoch': 10.0})

### testing on new words

In [25]:
def predict(word):
    inputs = tokenizer(word, return_tensors="pt")
    outputs = model.generate(**inputs, max_length=10)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [26]:
print(predict("dog"))

elfa


In [27]:
print(predict("apple"))

elma


In [ ]:
print(tokenizer.tokenize('dog'))

['▁dog']
